<a href="https://colab.research.google.com/github/beyzaturku/runway_detection_and_segmentation/blob/main/detect_and_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install ultralytics opencv-python
!pip install tensorflow
!pip install torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 126.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 97.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [3]:
from ultralytics import YOLO

yolo_model = YOLO("/content/drive/MyDrive/runway_detection_and_tracking/trained_models/gard/weights/best.pt")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [6]:
import tensorflow as tf
import torch

unet_model = torch.load("/content/drive/MyDrive/runway_detection_and_tracking/segmentation_train_models/unet_final.pth")

In [8]:
import torch.nn as nn

class UNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=1):
        super(UNet, self).__init__()

        # Encoder
        self.enc1 = self.conv_block(in_channels, 64)
        self.enc2 = self.conv_block(64, 128)
        self.enc3 = self.conv_block(128, 256)
        self.enc4 = self.conv_block(256, 512)

        # Bottleneck
        self.bottleneck = self.conv_block(512, 1024)

        # Decoder
        self.dec4 = self.conv_block(1024 + 512, 512)
        self.dec3 = self.conv_block(512 + 256, 256)
        self.dec2 = self.conv_block(256 + 128, 128)
        self.dec1 = self.conv_block(128 + 64, 64)

        self.final = nn.Conv2d(64, out_channels, 1)
        self.sigmoid = nn.Sigmoid()

    def conv_block(self, in_ch, out_ch):
        return nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        # Encoder path
        e1 = self.enc1(x)
        e2 = self.enc2(nn.MaxPool2d(2)(e1))
        e3 = self.enc3(nn.MaxPool2d(2)(e2))
        e4 = self.enc4(nn.MaxPool2d(2)(e3))

        # Bottleneck
        b = self.bottleneck(nn.MaxPool2d(2)(e4))

        # Decoder path with skip connections
        d4 = self.dec4(torch.cat([nn.Upsample(scale_factor=2)(b), e4], 1))
        d3 = self.dec3(torch.cat([nn.Upsample(scale_factor=2)(d4), e3], 1))
        d2 = self.dec2(torch.cat([nn.Upsample(scale_factor=2)(d3), e2], 1))
        d1 = self.dec1(torch.cat([nn.Upsample(scale_factor=2)(d2), e1], 1))

        return self.final(d1)

In [29]:
import cv2
import numpy as np
import torch
from ultralytics import YOLO
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

yolo_model = YOLO("/content/drive/MyDrive/runway_detection_and_tracking/trained_models/gard/weights/best.pt")

unet_model = UNet()
unet_model.load_state_dict(torch.load("/content/drive/MyDrive/runway_detection_and_tracking/segmentation_train_models/unet_final.pth", map_location=device))
unet_model.to(device)
unet_model.eval()

input_path = "/content/drive/MyDrive/runway_detection_and_tracking/input_videos/temiz_inis.mp4"
output_path = "/content/drive/MyDrive/runway_detection_and_tracking/output_videos/0806_test_19.mp4"

cap = cv2.VideoCapture(input_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'XVID'), fps, (width, height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # YOLO tespiti
    results = yolo_model(frame)
    boxes = results[0].boxes.xyxy.cpu().numpy() if results[0].boxes else []

    # 🔹 Segmentasyon HER ZAMAN yapılır
    resized = cv2.resize(frame, (256, 256)) / 255.0
    tensor_input = torch.tensor(resized.transpose(2, 0, 1)).unsqueeze(0).float().to(device)

    with torch.no_grad():
        mask_pred = unet_model(tensor_input).squeeze().cpu().numpy()

    mask_resized = cv2.resize(mask_pred, (width, height))
    mask_binary = (mask_resized > 0.3).astype(np.uint8) * 255

    # Morfolojik iyileştirme
    kernel = np.ones((7, 7), np.uint8)
    mask_binary = cv2.morphologyEx(mask_binary, cv2.MORPH_CLOSE, kernel)
    mask_binary = cv2.dilate(mask_binary, kernel, iterations=2)

    # Mavi maske
    blue_mask = np.zeros_like(frame)
    blue_mask[:, :, 0] = mask_binary
    overlay = cv2.addWeighted(frame, 0.6, blue_mask, 0.9, 0)

    # 🔹 Eğer bbox varsa: çiz
    if len(boxes) > 0:
        x1, y1, x2, y2 = map(int, boxes[0][:4])
        conf = results[0].boxes.conf[0]
        cv2.rectangle(overlay, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(overlay, "Pist", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # Çıktı yaz
    out.write(overlay)

cap.release()
out.release()
print("✅ Video tamamlandı:", output_path)


0: 384x640 (no detections), 12.5ms
Speed: 1.4ms preprocess, 12.5ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 11.3ms
Speed: 1.3ms preprocess, 11.3ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 11.1ms
Speed: 1.2ms preprocess, 11.1ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 10.9ms
Speed: 1.3ms preprocess, 10.9ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 10.9ms
Speed: 1.3ms preprocess, 10.9ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 10.9ms
Speed: 1.3ms preprocess, 10.9ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 11.1ms
Speed: 1.3ms preprocess, 11.1ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 11.1ms
Speed: 1.3ms preprocess, 11.1ms i

#### **HEDEF**


*   Polinom fit ile pist merkez hattını çizmek
*   Uçağın burnunu görüntüde varsayılan bir nokta olarak kabul edip (örneğin alt-orta)
*   Burnun pist merkezine olan yatay sapmasını ölçmek


#### **Pist Merkez Hattı Çizimi**

*   Üstten alta doğru giden (dikey) non-zero piksel satırları taranır
*   Her satırda sağ ve sol kenarlar bulunur
*   Ortası alınıp (x,y) merkez noktaları çıkartılır
*   Bu merkez noktalarına 2.dereceden polinom fit yapılır

In [32]:
import numpy as np

def fit_centerline_from_mask(mask_binary, step=10):
    height, width = mask_binary.shape
    center_points = []

    for y in range(0, height, step):
        x_indices = np.where(mask_binary[y, :] > 0)[0]
        if len(x_indices) > 2:
            x_center = int(np.mean(x_indices))
            center_points.append((x_center, y))

    if len(center_points) < 5:
        return None, []

    # Noktalardan polinom fit (2. derece)
    xs, ys = zip(*center_points)
    poly_fit = np.poly1d(np.polyfit(ys, xs, deg=2))  # x = f(y)
    return poly_fit, center_points

    uçak_burnu_x = width // 2
    uçak_burnu_y = height - 1

    pist_merkezi_x = int(poly_fit(uçak_burnu_y))
    sapma_px = abs(uçak_burnu_x - pist_merkezi_x)

In [33]:
def draw_centerline_and_deviation(frame, mask_binary, poly_fit):
    height, width = frame.shape[:2]
    frame_copy = frame.copy()

    # Merkez çizgisi çizimi
    for y in range(0, height, 5):
        x = int(poly_fit(y))
        if 0 <= x < width:
            cv2.circle(frame_copy, (x, y), 1, (0, 255, 255), -1)  # sarı nokta

    # Uçak burnu noktası
    uçak_x = width // 2
    uçak_y = height - 1
    pist_x = int(poly_fit(uçak_y))
    sapma = abs(uçak_x - pist_x)

    # Görsel çizim
    cv2.line(frame_copy, (uçak_x, uçak_y), (pist_x, uçak_y), (0, 0, 255), 2)
    cv2.circle(frame_copy, (uçak_x, uçak_y), 5, (255, 255, 255), -1)

    # Yazı
    cv2.putText(frame_copy, f"Sapma: {sapma}px", (30, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

    return frame_copy

In [34]:
# Segmentasyon maskesi alındıktan sonra:
poly_fit, center_pts = fit_centerline_from_mask(mask_binary)

if poly_fit is not None:
    overlay = draw_centerline_and_deviation(overlay, mask_binary, poly_fit)

### **BİRLEŞTİRİLMİŞ**

In [35]:
import cv2
import numpy as np
import torch
from ultralytics import YOLO

# 🔹 Cihaz seçimi
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 🔹 YOLO modeli (senin yoluna göre ayarla)
yolo_model = YOLO("/content/drive/MyDrive/runway_detection_and_tracking/trained_models/gard/weights/best.pt")

# 🔹 U-Net modeli yükle
unet_model = UNet()
unet_model.load_state_dict(torch.load(
    "/content/drive/MyDrive/runway_detection_and_tracking/segmentation_train_models/unet_final.pth",
    map_location=device))
unet_model.to(device)
unet_model.eval()

# 🔹 Giriş / Çıkış Video
input_path = "/content/drive/MyDrive/runway_detection_and_tracking/input_videos/temiz_inis.mp4"
output_path = "/content/drive/MyDrive/runway_detection_and_tracking/output_videos/0806_test_20.mp4"

cap = cv2.VideoCapture(input_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'XVID'), fps, (width, height))

# 🔹 Polinom fit fonksiyonu
def fit_centerline_from_mask(mask_binary, step=10):
    height, width = mask_binary.shape
    center_points = []

    for y in range(0, height, step):
        x_indices = np.where(mask_binary[y, :] > 0)[0]
        if len(x_indices) > 2:
            x_center = int(np.mean(x_indices))
            center_points.append((x_center, y))

    if len(center_points) < 5:
        return None, []

    xs, ys = zip(*center_points)
    poly_fit = np.poly1d(np.polyfit(ys, xs, deg=2))  # x = f(y)
    return poly_fit, center_points

# 🔹 Merkez çizgisi ve sapma çizim fonksiyonu
def draw_centerline_and_deviation(frame, mask_binary, poly_fit):
    height, width = frame.shape[:2]
    frame_copy = frame.copy()

    # Merkez çizgisi
    for y in range(0, height, 5):
        x = int(poly_fit(y))
        if 0 <= x < width:
            cv2.circle(frame_copy, (x, y), 1, (0, 255, 255), -1)  # sarı nokta

    # Sapma hesaplama (uçak burnu: alt merkez)
    uçak_x = width // 2
    uçak_y = height - 1
    pist_x = int(poly_fit(uçak_y))
    sapma = abs(uçak_x - pist_x)

    # Çizimler
    cv2.line(frame_copy, (uçak_x, uçak_y), (pist_x, uçak_y), (0, 0, 255), 2)
    cv2.circle(frame_copy, (uçak_x, uçak_y), 5, (255, 255, 255), -1)
    cv2.putText(frame_copy, f"Sapma: {sapma}px", (30, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

    return frame_copy

# 🔁 Kare kare işleme
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 🔹 YOLO tespiti
    results = yolo_model(frame)
    boxes = results[0].boxes.xyxy.cpu().numpy() if results[0].boxes else []

    # 🔹 U-Net segmentasyon
    resized = cv2.resize(frame, (256, 256)) / 255.0
    tensor_input = torch.tensor(resized.transpose(2, 0, 1)).unsqueeze(0).float().to(device)

    with torch.no_grad():
        mask_pred = unet_model(tensor_input).squeeze().cpu().numpy()

    # 🔹 Maske boyutlandırma ve iyileştirme
    mask_resized = cv2.resize(mask_pred, (width, height))
    mask_binary = (mask_resized > 0.3).astype(np.uint8) * 255
    kernel = np.ones((7, 7), np.uint8)
    mask_binary = cv2.morphologyEx(mask_binary, cv2.MORPH_CLOSE, kernel)
    mask_binary = cv2.dilate(mask_binary, kernel, iterations=2)

    # 🔹 Mavi maske overlay
    blue_mask = np.zeros_like(frame)
    blue_mask[:, :, 0] = mask_binary
    overlay = cv2.addWeighted(frame, 0.6, blue_mask, 0.9, 0)

    # 🔹 Bbox varsa çiz
    if len(boxes) > 0:
        x1, y1, x2, y2 = map(int, boxes[0][:4])
        conf = results[0].boxes.conf[0]
        cv2.rectangle(overlay, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(overlay, f"runway {conf:.2f}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # 🔹 Merkez çizgisi ve sapma
    poly_fit, center_pts = fit_centerline_from_mask(mask_binary)
    if poly_fit is not None:
        overlay = draw_centerline_and_deviation(overlay, mask_binary, poly_fit)

    out.write(overlay)

# 🔚 Kapat
cap.release()
out.release()
print("✅ Tam video pipeline başarıyla tamamlandı:", output_path)



0: 384x640 (no detections), 14.9ms
Speed: 1.6ms preprocess, 14.9ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 11.2ms
Speed: 1.4ms preprocess, 11.2ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 11.8ms
Speed: 1.4ms preprocess, 11.8ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 11.4ms
Speed: 1.5ms preprocess, 11.4ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 11.4ms
Speed: 1.4ms preprocess, 11.4ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 11.4ms
Speed: 1.5ms preprocess, 11.4ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 11.6ms
Speed: 1.4ms preprocess, 11.6ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 11.5ms
Speed: 1.4ms preprocess, 11.5ms i